# Question & Answer System
by ES

In [1]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import urllib
import sys
import os
import zipfile
import tarfile
import json 
import hashlib
import re
import itertools
import gensim
import string
import math
import nltk
import random
import csv
import io
from collections import Counter
from nltk import pos_tag, word_tokenize
from nltk.tree import Tree

C:\Users\ES\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\ES\Anaconda3\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
model = gensim.models.Word2Vec(sentences, min_count=1)
print(type(model))
# print(model.shape)

<class 'gensim.models.word2vec.Word2Vec'>


In [3]:
def toWordList(str):
    word_list = []
    words = str.split()
    for word in words:
        wordd = word.strip(',.?\'\")(').lower()
        word_list.append(wordd)
    return word_list

documents_list = {}
documents_count = {}
word_set_dic ={}
with open('documents.json','r') as f:
    documents = json.load(f)
    for dic in documents:
        word_set = set()
        para_list = []
        document_list = []
        for sentence in dic["text"]:
            word_count=Counter()
            word_list = toWordList(sentence)
            for word in word_list:
                word_count[word]+=1
                word_set.add(word)
            para_list.append(word_list)
            document_list.append(word_count)
        documents_list[dic["docid"]] = para_list
        documents_count[dic["docid"]] = document_list
        word_set_dic[dic["docid"]]=set(word_set)

# for key in documents_count:
#     print (documents_count[key])
#     print()

In [4]:
print(documents_count[410][0]['a'])

5


In [5]:
def getTF(docid,word_count):
    tfVector = []
    for word in word_set_dic[docid]:
        tfVector.append(word_count[word])
    return tfVector

def getIDF(question,docid):
    idfVector = []
    for word in word_set_dic[docid]:
        if word in question:
            count = 0
            for para in documents_list[docid]:
                if word in para:
                    count +=1
            if not count == 0:
                idfVector.append( float( len(documents_list[docid]) )/count )
            else:
                idfVector.append(0)
        else:
            idfVector.append(0)
    return idfVector

def getParagraph(question,docid):
    tfidf_list=[]
    for paraIndex in documents_count[docid]:
        tfVector = getTF(docid,paraIndex)
        idfVector = getIDF(question,docid)
        zip_list = list(zip(tfVector,idfVector))
        f = lambda x,y:x*y
        prod = sum(list(map(f,tfVector,idfVector)))
        cosine = math.sqrt(np.sum(np.square(tfVector)))*math.sqrt(np.sum(np.square(idfVector)))
        if not cosine == 0:
            tfidf_list.append(float(prod)/cosine)
        else:
            tfidf_list.append(0)
#     print (docid,np.argmax(tfidf_list),tfidf_list)
#     print(documents[docid]["text"][np.argmax(tfidf_list)])
    return documents[docid]["text"][np.argmax(tfidf_list)],np.argmax(tfidf_list)
        
    

In [9]:
with open('training.json','r') as f:
    training = json.load(f)

with open('devel.json','r') as f:
    develop = json.load(f)
    
#     develop_data = []
#     for question in develop:
#         question_word_list = toWordList(question["question"])
#         paragraph,index = getParagraph(question_word_list,question["docid"])
#         develop_data.append((question["question"],paragraph,index))
    
with open('testing.json','r') as f:
    testing = json.load(f)
#     answerParagraph = []
    test_data = []
    for question in testing:
        question_word_list = toWordList(question["question"])
        paragraph,index = getParagraph(question_word_list,question["docid"])
        print(question["question"])
        test_data.append((question["question"],paragraph,index))

Modern browser support standards-based and defacto what?
What do people typically call a web browser?
What is it called when content is changed from markup to an interactive document?
What platform is a browser used on?
When was Firefox released?
Who released the Internet Explorer browser?
When was the first browser created?
HTTP Secure is supported by what?
Who released Mosaic?
Who invented the first browser?
The Commission felt that the bundling undermined what?
The release of Internet Explorer started the first what?
What does IRC stand for?
Who investigated the bundling of the IE browser with Windows OS?
Which browser is the newest to enter the field?
When was the first beta release for Safari?
When did the European Commission say they would investigate the bundling?
What does ftp stand for?
Which company pays Firefox to make their search engine the default on their browser?
What does http stand for?
Features on a browser can range from minimal with little support to what?
When did

When did the torch arrive in Vietnam?
What was stopped due to protest interruptions?
How many torchbearers did not participate because of their concerns with Tibet?
The Chinese people were told to show patriotism in an orderly and what manner?
Though several torchbearers withdrew from the torch relay, the official website for China stated that Indian torch participants vowed to run for what?
Which French newspaper was accused of being biased by Chinese State press agency Xinhua?
What is blogger Điếu Cày's real name?
What was halted with the torch relay for future Olympics?
What was burned during these boycotts?
Who said the police acted on their own in taking the flags?
How many kilometers was the torch supposed to go through Taiwan?
Prior to the 2008 games, when did Malaysia last see an Olympic torch relay?
Who amassed in large scale against protesters?
When did the torch arrive in Saint Petersburg?
Along with Francesca Martinez, who decided to not carry the torch?
In San Francisco, w

Who is the Green World Foundation chairwoman who refused to participate in the relay?
A newspaper article criticized that there were not enough of what kind of person among the torchbearers?
The international torch relay was canceled for what event in June 2008?
When did the torch reach Sanya, Hainan?
Who was said to have given money to the Dalai Lama?
What is the name of the handicapped bearer of the torch?
Which city hosted the 2012 Summer Olympics?
The media was told that the Chinese supplied with shirts and transportation?
Which cathedral did Reporters Without Borders hang another protest banner?
What French newspaper did the Chinese media accuse of bias?
When did a protest in Delhi occur to protest against Chinese repression in Tibet?
Which newspaper asked the Chinese people to protest peacefully?
Who was the first bearer of the torch in Argentina?
Where was the only place Tibetan flags were permitted?
What is the name of the Syrian actress who was one to bear the torch?
Where was

What name did the Anglo-Saxons give their settlement first?
What's the name of the wall tower that was the first of its kind in England built for artillery purposes?
How many people between the ages of 30 and 34 live in Southampton?
How many decibels was John Melody able to reach in his cry?
What size ships can move through Southampton Water more easily because of the double high tide?
What kind of passenger ships make up a lot of the traffic in the port of Southampton?
When did Southampton officially become a city?
Which level of schools in Southampton are the worst behaved in the UK?
What specific form of bullying did the survey show was most common?
How many crew members out of every five on the RMS Titanic were Sotonians?
In what year did the rail route from Southampton to London open?
What year was the University of Southampton founded?
What company owns WestQuay Watermark?
In what decade did the Southern Railway Company commission the program that created the Western Docks?
What 

What's Southampton University's radio station called?
Out of about every 14 students, how many will be suspended from school?
What famous ballet company has performed at the Mayflower?
In what decade was the ferry port built in Southampton to carry people to exotic destinations on the continent?
In what month of 1940 did bombs destroy the factory that made Mitchell's seaplanes?
What town did Choules say had the only Main Street more beautiful than Southampton's?
How many concillors sit on Southampton's City Council?
What's the only monocentric city on the South Coast larger than Southampton?
What was the name of the man who overhauled block-making to mechanize the process?
What body of water is actually responsible for the unusual double high tide?
What sector puts up most of Southampton's economic contribution?
During the 13th century, what did Southampton get in trade for English cloth and wool?
Outside of what structure did the execution of the Southampton Plot leaders take place?
W

...

Which series of applications was Claris responsible for inventing?
When was the Power Mac G4 discontinued?
What did Claris rename the Informix Wingz spreadsheet program?
What ranking was Apple in the PC global market share during 2014?
What type of software began to emerge following the release of Intel-based Macs?
Which smartphone has hurt personal computer manufacterers the most?
What effect did Mac's decision to brand the Mac OS8 have on companies such as Motorola?
What year was Raskin hired by Apple?
What has been the best-selling ultra-portable over Windows Ultrabooks in the U.S.?
Where does Mac harware boot directly to in most PowerPC-based Macs?
What Macintosh first included an on-board MMU?
Who did the Macintosh clones hurt financially?
What are the top-selling form factors of the Mac platform today?
Who designed, develped, and marketed the Mac computers?
Who eventually lead the project on the Mac team?
What was the less expensive version of the Macintosh SE that was offered un

When did the era of planning start in Himachal Pradesh?
What is the Muslim population in Himachal Pradesh?
What kind of plan was it?
What occurs in the climatic conditions of Himachal?
What was the lowest class of Roman citizens?
What style of religious dedication was not uncommon for outer border areas?
Even in other places, of what did the legions keep observance?
What did a priesthood in the Imperial cult gain a provencial?
How was the sacrifice to the gods below handled?
For whose well being were public vows made in the empire?
What edict defined imperial ideas as being those of toleration?
Who paid for the most lavish of festival events?
What officer headed the religious personnel of a Roman camp?
What was the Council of Nicaea meant to define?
What did augers seek to understand through observances?
What did Valerian's second edict call the Christians' presence in the empire?
What was excessive religio equated with in Rome?
What act did Romans view as obnoxious?
What type of sacri

Who did the Senate select to settle a strike by the lower classes?
What did Augustus wish to do for Roman religion?
As a result of what war were many new temples built by victorious generals?
What type of diviners did the armies use to determine the will of the gods?
How did later Christians view Gladiatorial combats?
From what country did Erichtho come?
What religious group was exempt from sacrifices?
What person marked the religious area ritually?
What group viewed the mysteries as suspicious or subversive?
What vow was required of Vestals?
Which ancient hero brought the lares cult to Rome?
Around what aspect did the Romans a lot space?
To what Roman god was Dionysus similar?
What is an example of an event organized religious celebration?
What agreement was reached a the Council of Nicaea ?
Of what were Roman camps diminutive versions?
What did the ancient Romans expect after death?
How were the Vestals' devotion to Rome's security viewed to be ?
What type of campaign did Publius fig

Which US State did not  enact reception statues? 
Who signs a bill into a law?
When was the Greenman rule in Section 402A of the Restatement (Second) of Torts published?
What is a misdemeanor?
What kind of laws are not given statutory citation by the OFR?
What covers crimes of human against human?
In the Unites States what sets out the boundaries of federal law?
What does federal law come from?
Where are the majority of the U.S.'s crimes prosecuted?
How is a court hierarchy established?
What can delete a statute and make it unconstitutional?
What year was the case Greenman v. Yuba Power products?
By 1910, how many volumes of American reports were there?
States may grant their citizens border rights as long as they do not infringe on what?
How close is current American law to English law?
How many cases did appellate courts receice in 2010?
Once the slip laws are placed into the United States Statutes at Large, what are they called?
What did New York enact that replaced traditional comm

Which color was said to be the Emperor's color in ancient Rome?
On what date did the Garde Nationale kill as many as fifty people on Champs de Mars?
Red is opposite what color of the spectrum?
What is the cross on the banners used during the First Crusade known as?
What type of temper are people with red hair considered to have?
To whom did Rothko donate Passion of Christ murals?
What color did Matisse claim would impact ones blood pressure?
What kind of bural was at Adaoutse, Bouches-du-Rhône?
What color gowns are generally worn by recent Doctors of Philosophy at Oxford?
When was the Communist Manifesto first created?
What has to occur for a player in association football to be shown a red penalty card?
Christ and which other Christian figure were often painted red in medieval painting?
During what years did the Song dynasty occur?
Where did the Cincinnati Red Stockings move after leaving Cincinnati?
What type of palette was found in Tutankhamun's tomb?
The wealthiest in Europe used w

In [10]:
def addEntity(sentence):
    tagged = pos_tag(word_tokenize(sentence))
    entity = nltk.chunk.ne_chunk(tagged)
    return entity

In [35]:
def getEntity(data_list):
    for data in data_list:
        question_entity = addEntity(data[0])
        paragraph_entity = addEntity(data[1])

def getAnswer(data_list):
    answer=[]
    id_num = 0
    count = [0,0,0,0,0]
    useList = [0,0]
    for data in data_list:
        question_entity = addEntity(data[0])
        paragraph_entity = addEntity(data[1])
        answer_list = []
        
        if bool(re.search('what',data[0],re.IGNORECASE)):
            count[0]+=1
            for tree in paragraph_entity:
                try:
                    if tree[1] == "NN":
                        answer_list.append(tree[0])
                except:
                    pass
        elif bool(re.search('when',data[0],re.IGNORECASE)):
            count[1]+=1
            for tree in paragraph_entity:
                try:
                    if tree.label()=="TIME":
                        name = ""
                        for sub_tree in tree:
                            name.append(sub_tree[0]+" ")
                        answer_list.append(name[:-1])
                    if tree.label()=="DATE":
                        name = ""
                        for sub_tree in tree:
                            name.append(sub_tree[0]+" ")
                        answer_list.append(name[:-1])
                    if tree[1] == "CD":
                        answer_list.append(tree[0])
                except:
                    pass
        elif bool(re.search('who',data[0],re.IGNORECASE)):
            count[2]+=1
            for tree in paragraph_entity:
                try:
                    if tree.label()=="PERSON":
                        name = ""
                        for sub_tree in tree:
                            name.append(sub_tree[0]+" ")
                        answer_list.append(name[:-1])
                except:
                    pass
        elif bool(re.search('where',data[0],re.IGNORECASE)):
            count[3]+=1
            for tree in paragraph_entity:
                try:
                    if tree.label()=="LOCATION":
                        name = ""
                        for sub_tree in tree:
                            name.append(sub_tree[0]+" ")
                        answer_list.append(name[:-1])
                    if tree.label()=="gpe":
                        name = ""
                        for sub_tree in tree:
                            name.append(sub_tree[0]+" ")
                        answer_list.append(name[:-1])
                except:
                    pass
        else:
            count[4]+=1
            for tree in paragraph_entity:
                try:
                    if tree[1] == "NN":
                        answer_list.append(tree[0])
                except:
                    pass
        if len(answer_list)==0:
            useList[1]+=1
            index = random.randint(0,len(paragraph_entity)-1)
            try:
                answer.append (([str(id_num)+','+paragraph_entity[index][0].replace(",", "")],data[2]))
#                 print([str(id_num)+','+paragraph_entity[index][0].replace(",", "")],data[2])
            except:
                answer.append(([str(id_num)+','+paragraph_entity[index][0][0].replace(",", "")],data[2]))
#                 print([str(id_num)+','+paragraph_entity[index][0][0].replace(",", "")],data[2])
        else:
            useList[0]+=1
            index = random.randint(0,len(answer_list)-1)
#             print([str(id_num)+','+answer_list[index]],data[2])
            answer.append (([str(id_num)+','+answer_list[index].replace(",", "")],data[2]))
        id_num +=1
    print("count ",count)
    print("use list ",useList)
    return answer

    

# for line in answer:
#     print(line)
# count  [2198, 279, 422, 156, 563]
# use list  [2757, 861]

In [36]:
answer = getAnswer(test_data)
# answer = getAnswer(develop_data)

count  [2198, 279, 422, 156, 563]
use list  [2757, 861]


In [41]:
##Write into csv file
def writeFile(answer):
    # sys.stdout = io.TextIOWrapper(sys.stdout.buffer,encoding='utf8')
    out = open('answer.csv','a',newline='')
    csv_write =csv.writer(out,dialect='excel')
    csv_write.writerow(["id,answer"])
    answerID = 0
    for line in answer:
        print(line[0])
        try:
            csv_write.writerow(line[0])
        except:
            print("line "+str(answerID)+" cannot write")
        answerID+=1
    out.close()
    
writeFile(answer)


['0,development']
['1,source']
['2,https']
['3,browser']
['4,eventually']
['5,with']
['6,.']
['7,history']
['8,for']
['9,and']
['10,competition']
['11,browser']
['12,user']
["13,''"]
['14,founder']
['15,browser']
['16,product']
['17,browser']
['18,example']
['19,browser']
['20,news']
['21,the']
['22,expense']
['23,browser']
['24,year']
['25,feed']
['26,used']
['27,for']
['28,%']
['29,%']
['30,way']
['31,default']
['32,browser']
['33,information']
['34,experience']
['35,download']
['36,boom']
['37,web']
['38,addition']
['39,user']
['40,resource']
['41,use']
['42,folder']
['43,feed']
['44,2014']
['45,URIs']
['46,leader']
['47,']
['48,experience']
['49,']
['50,reader']
['51,industry']
['52,mailto']
['53,program']
['54,to']
["55,''"]
['56,claim']
['57,lady']
['58,torch']
['59,']
['60,community']
['61,torchbearer']
['62,citizen']
['63,report']
['64,fighter']
["65,''"]
['66,torch']
['67,with']
['68,relay']
['69,flame']
['70,woman']
['71,being']
['72,event']
['73,relay']
['74,.']
["75,''"]
['

['1686,control']
['1687,sun']
['1688,(']
['1689,name']
['1690,ambitious']
['1691,belligerence']
['1692,area']
['1693,and']
['1694,pain']
['1695,of']
['1696,occupation']
['1697,organization']
['1698,capital']
['1699,Jurchen']
['1700,uprising']
['1701,and']
['1702,of']
['1703,grace']
['1704,memoir']
['1705,throne']
['1706,Banner']
['1707,.']
['1708,suicide']
['1709,artillery']
['1710,from']
['1711,addition']
['1712,system']
['1713,sage']
['1714,Daoguang']
['1715,and']
['1716,artillery']
['1717,colonial']
['1718,long']
['1719,lack']
['1720,have']
['1721,achieved']
['1722,Russia']
['1723,月']
['1724,these']
['1725,Qing']
['1726,Qianlong']
['1727,crime']
['1728,use']
['1729,complexity']
['1730,manager']
['1731,term']
['1732,rule']
['1733,afternoon']
['1734,suit']
['1735,hour']
['1736,year']
['1737,end']
['1738,enthusiasts']
['1739,party']
['1740,zone']
['1741,idea']
['1742,clock']
['1743,change']
['1744,example']
['1745,fact']
['1746,timing']
['1747,trial']
['1748,zero']
['1749,time']
['1750

In [ ]:
from nltk import pos_tag, word_tokenize
text = "Hello my name is Derek. I live in Salt Lake city."
print(word_tokenize(text))
tagged = pos_tag(word_tokenize(text))
print(tagged)
entity = nltk.chunk.ne_chunk(tagged)
print(entity[0][1]=="NNP")
# entity.get('NN')
for tree in entity:
    try:
        print(tree.label()=="PERSON")
    except:
        pass
#     else:
#         pass
#     print(entity[4].label())
# entity.draw()
# t = Tree(Tree('a', Tree('b', Tree('c', Tree('d')))))
# print(t.kids.next.next.val)
# print(entity.kids.next.val)